## Imports

In [1]:
import pandas as pd
## pandas 1.1.1 necessary
import numpy as np
import string 
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import math
from gensim.models import Word2Vec
from plotly.graph_objects import Scatter3d

In [9]:
check = pd.read_pickle("../../delphes/data/extended_tweet_df")

In [11]:
check[check["country"]=="Czechia"]

,mep_id,name,country,group,nat_group,twitter,content
32522,124709,Martina DLABAJOVÁ,Czechia,Renew Europe Group,ANO 2011,Mdlabajova,Ve Zlíně to žije! Už po šedesáté! @ZlinFilmFes...
32523,124709,Martina DLABAJOVÁ,Czechia,Renew Europe Group,ANO 2011,Mdlabajova,Za designem do opuštěné funkcionalistické budo...
32524,124709,Martina DLABAJOVÁ,Czechia,Renew Europe Group,ANO 2011,Mdlabajova,@dudak_tomas Už se těším! 👌🏼
32525,124709,Martina DLABAJOVÁ,Czechia,Renew Europe Group,ANO 2011,Mdlabajova,Děkuji @david_klimes a @Aktualnecz za zajímavý...
32526,124709,Martina DLABAJOVÁ,Czechia,Renew Europe Group,ANO 2011,Mdlabajova,Výroba roušek a respirátorů z nanovlákna SpurT...
...,...,...,...,...,...,...,...
135795,124713,Tomáš ZDECHOVSKÝ,Czechia,Group of the European People's Party (Christia...,Křesťanská a demokratická unie - Československ...,TomasZdechovsky,Z Číny unikly vzácné záběry z lágru. Ujgur trá...
135796,124713,Tomáš ZDECHOVSKÝ,Czechia,Group of the European People's Party (Christia...,Křesťanská a demokratická unie - Československ...,TomasZdechovsky,Milá twistterová bublino - mám na Vás otázku: ...
135797,124713,Tomáš ZDECHOVSKÝ,Czechia,Group of the European People's Party (Christia...,Křesťanská a demokratická unie - Československ...,TomasZdechovsky,O dopravě slýcháme nyní hlavně v souvislosti s...
135798,124713,Tomáš ZDECHOVSKÝ,Czechia,Group of the European People's Party (Christia...,Křesťanská a demokratická unie - Československ...,TomasZdechovsky,Pavel Zeman sice nesouhlasí s rozhodnutím kraj...


In [24]:
data = pd.read_pickle("merger")

In [26]:
data["content_y"]

0        [False, [0.101227, -0.00355242, -0.0613298, 0....
1                                                     None
2        [[-0.0540806, 0.0168135, 0.0596281, 0.0681775,...
3        [[0.00612917, 0.0156791, -0.0486738, 0.0480299...
4        [False, [-0.0484475, 0.011017, -0.0737066, 0.0...
                               ...                        
11225    [[0.0495577, 0.129444, -0.0491547, 0.00429166,...
11226                                              [False]
11227    [[-0.0972181, -0.0736294, 0.00087396, 0.063221...
11228    [[-0.0196253, -0.00134424, -0.035015, -0.01173...
11229    [[-0.0603085, -0.00909752, -0.0233168, 0.00935...
Name: content_y, Length: 11230, dtype: object

In [22]:
data["content_y"] = data["content_y"].dropna()

## Adapting

In [235]:
extract = data[["index","content_y"]]

### Dropping the None

In [236]:
count = 0
dropables = []
for i in extract["content_y"]:
    if i == None:
        print(count)
        dropables.append(count)
        count+= 1
    else:
        count+=1

1
7
398
656
2157
2159
2220
2244
2269
2572
3246
3297
3539
4106
4509
4517
4519
4591
4713
4718
4735
4762
5072
5095
5097
5246
5634
5850
5852
5909
5954
6057
6061
6182
6221
7361
7663
7976
8308
8469
8856
9010
9014
10280
10384
10515
10817


In [237]:
extract = extract.drop(labels=dropables)

### Dropping the [False]

In [238]:
count = 0
falsables = []
for i in extract["content_y"]:
    if i == [False]:
        print(count)
        falsables.append(count)
        count+= 1
    else:
        count+=1

47
203
473
665
886
921
933


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [239]:
extract = extract.drop(labels=falsables)

In [240]:
extract = extract.reset_index()

In [241]:
extract = extract[["level_0","index","content_y"]]

In [242]:
extract

,level_0,index,content_y
0,0,5076,"[False, [0.101227, -0.00355242, -0.0613298, 0...."
1,2,5078,"[[-0.0540806, 0.0168135, 0.0596281, 0.0681775,..."
2,3,5079,"[[0.00612917, 0.0156791, -0.0486738, 0.0480299..."
3,4,5080,"[False, [-0.0484475, 0.011017, -0.0737066, 0.0..."
4,5,5081,"[[-0.0989527, 0.0149615, 0.0241871, 0.0582589,..."
...,...,...,...
11171,11225,133995,"[[0.0495577, 0.129444, -0.0491547, 0.00429166,..."
11172,11226,133996,[False]
11173,11227,133997,"[[-0.0972181, -0.0736294, 0.00087396, 0.063221..."
11174,11228,133998,"[[-0.0196253, -0.00134424, -0.035015, -0.01173..."


In [243]:
tweets = []
check = []
dict_tweets = {}
for i in extract["level_0"]:
    result = 0
    prov_results = []
    if len(extract["content_y"][i]) > 1: 
        for j in range (0,len(extract["content_y"][i])): 
            if extract["content_y"][i] != False: 
                prov_results.append(extract["content_y"][i][j])
            else:
                pass
        dict_tweets[extract["index"][i]] = (sum(prov_results)/len(extract["content_y"][i]))
        check.append(i)
    else: 
        pass
        check.append(i)
        
            

KeyError: 11176

### Deleting Floats

In [245]:
floats = []
for i,j in dict_tweets.items(): 
    if type(j) != np.ndarray:
        floats.append(i)

In [247]:
for i in floats:
    del dict_tweets[i]

### Creating DF

In [252]:
d_i_tweets = pd.DataFrame(dict_tweets).transpose()

In [263]:
d_i_tweets = d_i_tweets.reset_index()

### Color and Stuff

In [261]:
data = data.set_index("index")

In [280]:
da_shit = data.merge(d_i_tweets,on="index")

In [281]:
da_shit

,index,mep_id,name,country,group,nat_group,twitter,content_x,content_y,0,...,290,291,292,293,294,295,296,297,298,299
0,5076,28161,Margrete AUKEN,Denmark,Group of the Greens/European Free Alliance,Socialistisk Folkeparti,MargreteAuken,Hvornår agerer #Bagmandspolitiet på anmeldelse...,"[False, [0.101227, -0.00355242, -0.0613298, 0....",-0.016846,...,0.010223,-0.011244,0.009240,0.028963,-0.008443,-0.042849,-0.014880,0.041051,0.023303,-0.012236
1,5079,28161,Margrete AUKEN,Denmark,Group of the Greens/European Free Alliance,Socialistisk Folkeparti,MargreteAuken,"Amazonas brænder, og skov- og klimakrisen er a...","[[0.00612917, 0.0156791, -0.0486738, 0.0480299...",-0.025288,...,0.014924,-0.009034,0.017069,0.036944,0.000622,-0.024272,-0.017961,0.043717,0.026246,-0.006339
2,5080,28161,Margrete AUKEN,Denmark,Group of the Greens/European Free Alliance,Socialistisk Folkeparti,MargreteAuken,Covid-19 vaccine makers lobby the EU for immun...,"[False, [-0.0484475, 0.011017, -0.0737066, 0.0...",-0.026789,...,0.033633,-0.010781,0.009733,-0.002281,0.006047,-0.046107,-0.023379,0.041657,0.028609,-0.009291
3,5081,28161,Margrete AUKEN,Denmark,Group of the Greens/European Free Alliance,Socialistisk Folkeparti,MargreteAuken,Den britiske økonomi er i knæ: 'De kommende må...,"[[-0.0989527, 0.0149615, 0.0241871, 0.0582589,...",-0.018487,...,0.008235,-0.011501,0.014615,0.038758,0.009990,-0.015177,-0.028269,0.005696,0.034646,0.007942
4,5082,28161,Margrete AUKEN,Denmark,Group of the Greens/European Free Alliance,Socialistisk Folkeparti,MargreteAuken,"Jeg bliver så fly forbandet vred, ked af det, ...","[[-0.0496591, -0.0732693, 0.0299527, 0.0747977...",-0.010963,...,0.018865,-0.011564,0.013905,0.023749,0.010506,-0.017543,-0.027306,0.027216,0.015570,0.010784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10673,133994,5392,Lara WOLTERS,Netherlands,Group of the Progressive Alliance of Socialist...,Partij van de Arbeid,larawoltersEU,A Green New Deal for Europe must include great...,"[[0.0495577, 0.129444, -0.0491547, 0.00429166,...",-0.029210,...,0.037759,-0.015325,0.009431,-0.009702,0.009091,-0.043326,-0.025808,0.038204,0.031567,-0.006687
10674,133995,5392,Lara WOLTERS,Netherlands,Group of the Progressive Alliance of Socialist...,Partij van de Arbeid,larawoltersEU,A more socially and environmentally sustainabl...,"[[0.0495577, 0.129444, -0.0491547, 0.00429166,...",-0.041173,...,0.017315,0.016870,0.006894,-0.020515,0.003003,-0.046619,-0.022752,0.059461,0.049146,-0.007352
10675,133997,5392,Lara WOLTERS,Netherlands,Group of the Progressive Alliance of Socialist...,Partij van de Arbeid,larawoltersEU,Goed gesprek met @EUombudsman Emily O’Reilly o...,"[[-0.0972181, -0.0736294, 0.00087396, 0.063221...",-0.030016,...,0.022029,-0.017187,0.017818,-0.004375,-0.000672,-0.035493,-0.011981,0.056631,0.027491,-0.012153
10676,133998,5392,Lara WOLTERS,Netherlands,Group of the Progressive Alliance of Socialist...,Partij van de Arbeid,larawoltersEU,@pieterzwaan Ja hoor! Graag naar mijn EP email.,"[[-0.0196253, -0.00134424, -0.035015, -0.01173...",-0.006542,...,0.029530,0.007000,-0.000413,-0.044168,0.037959,-0.042401,-0.030223,0.052912,0.042841,-0.019456


In [266]:
def colors(values):
    if values == "Netherlands":
        return 0
    elif values == "Denmark":
        return 1
    else:
        return 2

In [276]:
def groups(values):
    if values == 'Group of the Greens/European Free Alliance':
        return 0
    elif values == 'Renew Europe Group':
        return 1
    elif values == "Group of the European People's Party (Christian Democrats)":
        return 2
    elif values == 'Group of the Progressive Alliance of Socialists and Democrats in the European Parliament':
        return 3
    elif values == 'European Conservatives and Reformists Group':
        return 4
    elif values == 'Group of the European United Left - Nordic Green Left':
        return 5
    elif values == 'Non-attached Members':
        return 6
    else:
        return 7

In [294]:
parties_list = ['Socialistisk Folkeparti',
       'Volkspartij voor Vrijheid en Democratie',
       'Christen Democratisch Appèl', 'Partij van de Arbeid',
       'ChristenUnie', 'GroenLinks', 'Forum voor Democratie',
       'Socialdemokratiet', 'Partij voor de Dieren', 'Democraten 66',
       'Venstre, Danmarks Liberale Parti', 'Det Radikale Venstre',
       'GO Realisme & Daadkracht', 'Staatkundig Gereformeerde Partij',
       'Enhedslisten', 'Det Konservative Folkeparti']

def parties(values):
    for i in parties_list:
        if values == i:
            return parties_list.index(i)

In [295]:
da_shit["nat_party_code"] = da_shit["nat_group"].apply(parties)

In [297]:
da_shit["nat_party_code"].to_csv("nat_parties.tsv",sep="\t", index=False, header=False)

In [282]:
da_shit["group_code"] = da_shit["group"].apply(groups)

In [283]:
da_shit["group_code"].to_csv("groups.tsv",sep="\t", index=False, header=False)

In [300]:
da_shit["color"] = da_shit["country"].apply(colors)

In [270]:
da_shit["color"].to_csv("countries.tsv",sep="\t", index=False, header=False)

In [303]:

da_shit[["color","group_code","nat_party_code"]].to_csv("multi_variables.tsv",sep="\t", index=False, header=True)



In [305]:
czech =  pd.read_pickle("czech")

In [306]:
czech = pd.DataFrame(czech)

In [307]:
czech

,content
32522,"[False, False, False, False, False, False, False]"
32523,"[[-0.0995356, -0.047137, 0.0250813, 0.0260645,..."
32524,"[False, False, False, False]"
32525,"[False, [-0.0120511, -0.0191111, -0.0630558, 0..."
32526,"[False, False, [-0.0120511, -0.0191111, -0.063..."
...,...
135795,"[[-0.065231, -0.0981508, 0.0377665, 0.051571, ..."
135796,"[[-0.0227174, 0.00630098, -0.0904016, 0.037196..."
135797,"[[0.0374981, -0.027252, -0.00474948, 0.0105297..."
135798,"[[-0.0882074, -0.0202498, 0.0247401, 0.0118748..."


### Saving

In [212]:
d_i_tweets.to_csv('d_i_tweets.tsv',sep="\t", index=False, header=False)

In [215]:
d_i_tweets

,index,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,0,-0.016846,-0.011860,-0.042851,0.044163,-0.027453,0.022471,0.000556,-0.036933,-0.001978,...,0.010223,-0.011244,0.009240,0.028963,-0.008443,-0.042849,-0.014880,0.041051,0.023303,-0.012236
1,1,-0.025288,-0.020077,-0.029960,0.033417,-0.022732,0.031891,0.014065,-0.054808,0.008609,...,0.014924,-0.009034,0.017069,0.036944,0.000622,-0.024272,-0.017961,0.043717,0.026246,-0.006339
2,2,-0.026789,0.006986,-0.025454,0.034253,-0.019140,0.003891,-0.009987,-0.062290,-0.005792,...,0.033633,-0.010781,0.009733,-0.002281,0.006047,-0.046107,-0.023379,0.041657,0.028609,-0.009291
3,3,-0.018487,-0.005300,-0.019736,0.044298,-0.020414,0.032369,0.003270,-0.053760,-0.012529,...,0.008235,-0.011501,0.014615,0.038758,0.009990,-0.015177,-0.028269,0.005696,0.034646,0.007942
4,4,-0.010963,-0.039525,-0.020190,0.044142,-0.027780,0.032248,0.009840,-0.049892,0.001871,...,0.018865,-0.011564,0.013905,0.023749,0.010506,-0.017543,-0.027306,0.027216,0.015570,0.010784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10673,10673,-0.029210,0.011155,-0.006377,0.046776,-0.025457,0.002079,0.006012,-0.028870,0.015737,...,0.037759,-0.015325,0.009431,-0.009702,0.009091,-0.043326,-0.025808,0.038204,0.031567,-0.006687
10674,10674,-0.041173,0.010350,-0.021792,0.045523,-0.020995,0.017644,0.002863,-0.055886,-0.019077,...,0.017315,0.016870,0.006894,-0.020515,0.003003,-0.046619,-0.022752,0.059461,0.049146,-0.007352
10675,10675,-0.030016,0.003368,-0.033410,0.023910,-0.019755,0.018780,0.005391,-0.040908,-0.017813,...,0.022029,-0.017187,0.017818,-0.004375,-0.000672,-0.035493,-0.011981,0.056631,0.027491,-0.012153
10676,10676,-0.006542,0.005297,-0.014884,0.020711,-0.029233,0.003898,-0.024561,-0.053010,-0.021391,...,0.029530,0.007000,-0.000413,-0.044168,0.037959,-0.042401,-0.030223,0.052912,0.042841,-0.019456
